# Multi part Assembly

### model & data loader setting

In [1]:
cd /data/wlsgur4011/part_assembly/src/multi-part-assembly-original

/data/wlsgur4011/part_assembly/src/multi-part-assembly-original


In [2]:
import os
import sys
import pwd
import argparse
import importlib

import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

from multi_part_assembly_original.datasets import build_dataloader
from multi_part_assembly_original.models import build_model
from multi_part_assembly_original.utils import PCAssemblyLogCallback

import multi_part_assembly_original

In [3]:
import sys,os,argparse
sys.argv = ["scripts/train.py" ,"--cfg_file" ,"configs/global/global-32x1-cosine_200e-artifact.py" ,"--fp16"]

In [10]:
parser = argparse.ArgumentParser(description='Training script')
parser.add_argument('--cfg_file', required=True, type=str, help='.py')
parser.add_argument('--category', type=str, default='', help='data subset')
parser.add_argument('--gpus', nargs='+', default=[0], type=int)
parser.add_argument('--weight', type=str, default='', help='load weight')
parser.add_argument('--fp16', action='store_true', help='FP16 training')
parser.add_argument('--cudnn', action='store_true', help='cudnn benchmark')
parser.add_argument('--vis', action='store_true', help='visualize results')
args = parser.parse_args()


sys.path.append(os.path.dirname(args.cfg_file))
cfg = importlib.import_module(os.path.basename(args.cfg_file)[:-3])
cfg = cfg.get_cfg_defaults()
cfg.data.overfit = 100
model = build_model(cfg)
model = model.cuda()

# Initialize dataloaders
train_loader, val_loader = build_dataloader(cfg)

In [11]:
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

### run!

In [14]:
from jhutil import to_cuda

for data in train_loader:
    data = to_cuda(data)
    
    import jhutil; jhutil.jhprint(1111, data)
    loss_dict = model.forward_pass(data, mode='test', optimizer_idx=-1)
    
    import jhutil; jhutil.jhprint(2222, loss_dict)
    break

1111  {
    "part_pcs": "tensor[32, 20, 1000, 3] n=1920000 (7.3Mb) x∈[-0.644, 0.560] μ=-3.477e-12 σ=0.050 cuda:0",
    "part_quat": "tensor[32, 20, 4] n=2560 (10Kb) x∈[-0.661, 0.993] μ=0.054 σ=0.244 cuda:0",
    "part_trans": "tensor[32, 20, 3] n=1920 (7.5Kb) x∈[-0.454, 0.486] μ=0.015 σ=0.095 cuda:0",
    "part_valids": "tensor[32, 20] n=640 (2.5Kb) x∈[0., 1.000] μ=0.250 σ=0.433 cuda:0",
    "data_id": "tensor[32] i64 x∈[8, 96] μ=51.188 σ=26.517 cuda:0",
    "instance_label": "tensor[32, 20, 0] n=0 \u001bempty\u001b cuda:0",
    "part_label": "tensor[32, 20, 0] n=0 \u001bempty\u001b cuda:0",
    "part_ids": "tensor[32, 20] n=640 (2.5Kb) x∈[0., 14.000] μ=0.806 σ=2.107 cuda:0"
}
2222  {
    "trans_loss": "tensor grad MeanBackward0 cuda:0 0.110",
    "rot_pt_cd_loss": "tensor grad MeanBackward0 cuda:0 0.012",
    "transform_pt_cd_loss": "tensor grad MeanBackward0 cuda:0 0.040",
    "rot_loss": "tensor grad MeanBackward0 cuda:0 0.584",
    "rot_pt_l2_loss": "tensor grad MeanBackward0 cuda: